In [4]:
import foolbox
import keras
import numpy as np
from keras.applications.resnet50 import ResNet50
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import cv2
import PIL.Image
import urllib

# instantiate the model
keras.backend.set_learning_phase(0)
kmodel = ResNet50(weights='imagenet')
preprocessing = (np.array([104,116,123]),1)
fmodel = foolbox.models.KerasModel(kmodel, bounds=(0,255), preprocessing=preprocessing)

# get source image and label
# download the image
#image, label = foolbox.utils.imagenet_example()

page = requests.get("http://image-net.org/api/text/imagenet.synset.geturls?wnid=n02123394")

#print(page.content)

# BeautifulSoup is an HTML parsing library

soup = BeautifulSoup(page.content, 'html.parser') #puts the content of the website into the soup variable, each url on a different line

str_soup = str(soup) # convert soup to string to be split
type(str_soup)
split_urls = str_soup.split('\r\n') # split so each url is a different position on a list
print(len(split_urls)) # to know how many image we have


1661


In [3]:
bikes_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n02834778")#bicycle synset
#print(bikes_page.content)
# BeautifulSoup is an HTML parsing library
bikes_soup = BeautifulSoup(bikes_page.content, 'html.parser')#puts the content of the website into the soup variable, each url on a different line
bikes_str_soup=str(bikes_soup)#convert soup to string so it can be split
type(bikes_str_soup)
bikes_split_urls=bikes_str_soup.split('\r\n')#split so each url is a different possition on a list
print(len(bikes_split_urls))

1345


In [ ]:
!mkdir /content/train #create the Train folder
!mkdir /content/train/pcat #create the persian cat folder
!mkdir /content/train/bikes #create the bikes folder
!mkdir /content/validation
!mkdir /content/validation/pcat #create the persian cat folder
!mkdir /content/validation/bikes #create the bikes folder

In [ ]:
img_rows, img_cols = 32, 32 #number of rows and columns to convert the images to
input_shape = (img_rows, img_cols, 3)#format to store the images (rows, columns,channels) called channels last

def url_to_image(url):
 # download the image, convert it to a NumPy array, and then read
 # it into OpenCV format
 resp = urllib.request.urlopen(url)
 image = np.asarray(bytearray(resp.read()), dtype="uint8")
 image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
 # return the image
 return image

n_of_training_images=100 #the number of training images to use

for progress in range(n_of_training_images):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not split_urls[progress] == None:
      try:
        I = url_to_image(split_urls[progress])
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/train/pcat/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)
      except:
        None
#do the same for bikes:
for progress in range(n_of_training_images):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not bikes_split_urls[progress] == None:
      try:
        I = url_to_image(bikes_split_urls[progress])
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/train/bikes/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)
      except:
        None
        
        
#Validation data:
for progress in range(50):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not split_urls[progress] == None:
      try:
        I = url_to_image(split_urls[n_of_training_images+progress])#get images that are different from the ones used for training
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/validation/pcat/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)
      except:
        None
#do the same for bikes:
for progress in range(50):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not bikes_split_urls[progress] == None:
      try:
        I = url_to_image(bikes_split_urls[n_of_training_images+progress])#get images that are different from the ones used for training
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/validation/bikes/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)
      except:
        None
        
print("\nTRAIN:\n")          
print("\nlist the files inside pcat directory:\n")        
!ls /content/train/pcat #list the files inside pcat
print("\nlist the files inside bikes directory:\n")
!ls /content/train/bikes #list the files inside bikes
print("\nVALIDATION:\n")
print("\nlist the files inside pcat directory:\n")        
!ls /content/validation/pcat #list the files inside pcat
print("\nlist the files inside bikes directory:\n")
!ls /content/validation/bikes #list the files inside bikes

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# get data route
base_dir = '/content'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_pcat_dir = os.path.join(train_dir, 'pcat')            # get persian cat train dir
train_bike_dir = os.path.join(train_dir, 'bikes')           # get bike train dir
validation_pcat_dir = os.path.join(validation_dir, 'pcat')  # get persian cat validation dir
validation_bike_dir = os.path.join(validation_dir, 'bikes') # get persian bike validation dir

train_pcat_fnames = os.listdir(train_pcat_dir)              # get persian cat train imgfile
train_bike_fnames = os.listdir(train_bike_dir)              # get bike train imgfile
validation_pcat_fnames = os.listdir(validation_pcat_dir)    # get persian cat validation imgfile
validation_bike_fnames = os.listdir(validation_bike_dir)    # get bike validation imgfile

# set row & col to 4
nrows = 4
ncols = 4

# set picture index
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_pcat_pix = [os.path.join(train_pcat_dir, fname) 
                for fname in train_pcat_fnames[pic_index-8:pic_index]]
next_bike_pix = [os.path.join(train_bike_dir, fname) 
                for fname in train_bike_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_pcat_pix+next_bike_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') 

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()